In [ ]:
import pulp

# Define food data dynamically
foods = {
    'Bread': {'cost': 2},
    'Milk': {'cost': 3},
    'Eggs': {'cost': 4}
}

# Define nutritional constraints dynamically
constraints = {
    'Calories': {'coefficients': {'Bread': 4, 'Milk': 3, 'Eggs': 2}, 'min': 10, 'max': None},
    'Protein': {'coefficients': {'Bread': 4, 'Milk': 8, 'Eggs': 7}, 'min': 15, 'max': None},
    'Fat': {'coefficients': {'Bread': 1, 'Milk': 5, 'Eggs': 5}, 'min': None, 'max': 10}
}

# Create problem
problem = pulp.LpProblem("Diet Optimization", pulp.LpMinimize)

# Create decision variables dynamically
food_vars = {food: pulp.LpVariable(food, lowBound=0, cat='Continuous') for food in foods}

# Add objective function dynamically
problem += pulp.lpSum([foods[food]['cost'] * food_vars[food] for food in foods]), "Total Cost"

# Add constraints dynamically
for constraint_name, constraint_data in constraints.items():
    coeffs = constraint_data['coefficients']
    expr = pulp.lpSum([coeffs[food] * food_vars[food] for food in foods])
    
    if constraint_data['min'] is not None:
        problem += expr >= constraint_data['min'], f"{constraint_name}_min"
    if constraint_data['max'] is not None:
        problem += expr <= constraint_data['max'], f"{constraint_name}_max"

# Solve the problem
problem.solve()



Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/musel/Documents/github/cutting-stock/.venv/lib/python3.13/site-packages/pulp/apis/../solverdir/cbc/linux/i64/cbc /tmp/4610132a31bb46c2a303fbf0126e78b9-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /tmp/4610132a31bb46c2a303fbf0126e78b9-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 8 COLUMNS
At line 21 RHS
At line 25 BOUNDS
At line 26 ENDATA
Problem MODEL has 3 rows, 3 columns and 9 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 3 (0) rows, 3 (0) columns and 9 (0) elements
0  Obj 0 Primal inf 4.3749998 (2)
2  Obj 6.5
Optimal - objective value 6.5
Optimal objective 6.5 - 2 iterations time 0.002
Option for printingOptions changed from normal to all
Total time (CPU seconds):       0.00   (Wallclock seconds):       0.00



/home/musel/Documents/github/cutting-stock/.venv/lib/python3.13/site-packages/pulp/pulp.py:1489: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


1

In [3]:
# Print the results
print("Status:", pulp.LpStatus[problem.status])
print("Optimal quantities:")
for variable in problem.variables():
    print(f"{variable.name}: {variable.varValue}")


Status: Optimal
Optimal quantities:
Bread: 1.75
Eggs: 0.0
Milk: 1.0


In [6]:
# Write the results to a file
problem.writeLP("DietOptimization.lp")

# Write to MPS format
problem.writeMPS("DietOptimization.mps")

[Bread, Eggs, Milk]

In [ ]:
for name, constraint in problem.constraints.items():
    print(f"{name}: {constraint}")
    print(f"  Slack: {constraint.slack}, Duel values: {constraint.pi}")

Calories: 4*Bread + 2*Eggs + 3*Milk >= 10
  Slack: -0.0, Shadow Price: 0.2
Protein: 4*Bread + 7*Eggs + 8*Milk >= 15
  Slack: -0.0, Shadow Price: 0.3
Fat: Bread + 5*Eggs + 5*Milk <= 10
  Slack: 3.25, Shadow Price: 0.0


In [8]:
# Reduced costs
for variable in problem.variables():
    print(f"{variable.name}: Reduced Cost: {variable.dj}")  
    

Bread: Reduced Cost: 0.0
Eggs: Reduced Cost: 1.5
Milk: Reduced Cost: 0.0
